In [1]:
import configparser
import dataclasses
import os
import re
import shutil
import statistics
from collections import Counter
from itertools import chain
from pathlib import Path
from time import perf_counter

import pandas as pd
from Bio import SeqIO, pairwise2
from Bio.pairwise2 import format_alignment
from Bio.Seq import Seq
from IPython.display import display
import logging

import utils
from fns import *

In [2]:
Path('logs').mkdir(parents=True, exist_ok=True)
log = utils.make_logger("logs/rp_log")
log.info("\nSTART")
log.info("\nCHECKING CONFIGURATION FILE CONTENTS...")

In [3]:
config = configparser.ConfigParser()
config.read_file(open('config.ini'))

In [4]:
# checking config file
# if any error occurs then program terminates; then check the config file

abi_sequence_folder = checking_dirs(config['Paths']['abi_sequence_folder'], log, log_msg=True, create_dir=False)
vh_template_sequence_folder = checking_dirs(config['Paths']['vh_template_sequence_folder'], log, log_msg=True, create_dir=False)
vl_template_sequence_folder = checking_dirs(config['Paths']['vl_template_sequence_folder'], log, log_msg=True, create_dir=False)
results_dir = checking_dirs(config['Paths']['results_dir'], log, log_msg=True, create_dir=True)
h3_nt_data_sheet_filepath = check_files(config['Files']['h3_nt_data_sheet_filepath'], log)
df = check_tsv_file(h3_nt_data_sheet_filepath, log)
excel_path_file_name = create_results_excel_file_path(config['Paths']['results_dir'], config['Files']['output_excel_file_name'])

# patters
pat_vh, pat_vl = get_patterns(config)
patrm_vh, patrm_vl = get_patterns_to_rm(config)

# creating dirs for copying matched abi files
res_dir_vh = checking_dirs(f"{results_dir}/{pat_vh}", log, log_msg=True, create_dir=True)
res_dir_vl = checking_dirs(f"{results_dir}/{pat_vl}", log, log_msg=True, create_dir=True)


In [5]:
log.info("\nCHECKING THE AB1 FILES...")
vh_abi_dict = {i.name.replace('.abi', '').replace(patrm_vh, '') : str(i) for i in sorted([*Path(abi_sequence_folder).glob(f"*{pat_vh}*.abi")])}
vl_abi_dict = {i.name.replace('.abi', '').replace(patrm_vl, '') : str(i) for i in sorted([*Path(abi_sequence_folder).glob(f"*{pat_vl}*.abi")])}

if len(vh_abi_dict) == len(vl_abi_dict):
    log.info(f"[+] INFO: There are {len(vh_abi_dict)} abi files in the abi sequence folder")
else:
    log.warning(f"[+] WARN: There is a differrence in number of abi files in vh: {len(vh_abi_dict)} and vl {len(vl_abi_dict)}")

In [6]:
sample_ids = checking_ab1_files(log, vh_abi_dict, vl_abi_dict)

In [7]:
log.info("\nCHECKING THE GENBANK FILES...")
vh_template_gb_dict =  {i.name.replace('.gb', '') : str(i) for i in sorted(Path(vh_template_sequence_folder).glob('*.gb'))}
vl_template_gb_dict =  {i.name.replace('.gb', '') : str(i) for i in sorted(Path(vl_template_sequence_folder).glob('*.gb'))}

if len(vh_template_gb_dict) == len(vl_template_gb_dict):
    log.info(f"[+] INFO: There are {len(vh_template_gb_dict)} genbank files in each dir vh and vl")
else:
    log.warning(f"[+] WARN: There is a differrence in number of genbank files in vh: {len(vh_template_gb_dict)} and vl {len(vl_template_gb_dict)}")

In [8]:
# dictionary containing probe seq and name
h3_dict = df.set_index('name', drop=True).to_dict().get('h3_nt')

In [9]:
# this loop  iternate through all the sample ids and find_match_on_all_h3probes function search for the probe on the normal seq and the rev complemented seq
# if there is a match which will be returened as a list

log.info(f"\nITERATING THROUGH EACH SAMPLE ID")
result_vh, result_vl = [], []
for sample in sample_ids:
    # print(f">>{sample}")
    _vhabi, _vlabi = get_abi_file_path(key=sample, vh_abi_dict=vh_abi_dict, vl_abi_dict=vl_abi_dict)
    vh_d, vh_dtrim, vh_dqtrimlst = get_seqobj_from_abi(_vhabi)
    vl_d, vl_dtrim, vl_dqtrimlst = get_seqobj_from_abi(_vlabi)
    
    # matching each probe on  VH and VL - normal and revcomp sequence
    vh_prob_search = find_match_on_all_h3probes(log, h3_dict, vh_d, sample, vh_abi_dict, vl_abi_dict)
    vl_prob_search = find_match_on_all_h3probes(log, h3_dict, vl_d, sample, vh_abi_dict, vl_abi_dict)
    if len(vh_prob_search) >=1:
        result_vh.append(vh_prob_search)
    if len(vl_prob_search) >=1:
        result_vl.append(vl_prob_search)
log.info(f"\nFINISH ITERATING THROUGH EACH SAMPLE ID")
# if vh and vl ids match then copy the files to a new place
colnames=["Match","h3_name","sample_id","vh_abi_fp","vl_abi_fp","probe_seq", "abi_seq", "seq_record"]
df_vh = pd.DataFrame(chain.from_iterable(result_vh))
df_vl = pd.DataFrame(chain.from_iterable(result_vl))
df_vh.columns, df_vl.columns = colnames, colnames

In [10]:
# if the sahpe of df_vh and df_vl are same and the h3_name in both dataframes are same then copy the matched data to a new dir
if(df_vl.shape == df_vh.shape) and (df_vl.h3_name == df_vh.h3_name).all():
    log.info(f"[+] INFO: There are {df_vl.shape[0]} matches in the dataframe between vh and vl")
    log.info(f"[+] INFO: Of which {df_vh[['vh_abi_fp', 'vl_abi_fp']].drop_duplicates().shape[0]} samples has to be moved to a new dir")
else:
    log.info(f"[+] INFO: There are {df_vh.shape[0]} matches in df_vh")
    log.info(f"[+] INFO: There are {df_vl.shape[0]} matches in df_vl")

In [11]:
df_vl.head()

,Match,h3_name,sample_id,vh_abi_fp,vl_abi_fp,probe_seq,abi_seq,seq_record
0,Forward_Strand_Match,TMH577-hF-014-G03,TMH577-hIgG1-013-A3_A03,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCAAGAGGAATTCGCCGTATTACTATGGTTCGGGGAGCTGGGGGGT...,"(A, C, G, G, A, A, G, C, C, G, G, A, A, G, C, ...","(A, C, G, G, A, A, G, C, C, G, G, A, A, G, C, ..."
1,Forward_Strand_Match,TMH577-hF-012-F03,TMH577-hIgG1-013-D10,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCGAGAGGGGGCGGATACAGCTATGGCATTGACTAC,"(G, G, G, C, A, T, G, C, C, T, G, G, A, G, G, ...","(G, G, G, C, A, T, G, C, C, T, G, G, A, G, G, ..."
2,Forward_Strand_Match,TMH577-hF-012-C01,TMH577-hIgG1-013-E11,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCGCGGGTGGGAGCCGCGGCCGATGCTTTTGATATC,"(T, G, G, C, A, G, T, C, C, T, G, A, G, C, A, ...","(T, G, G, C, A, G, T, C, C, T, G, A, G, C, A, ..."
3,Forward_Strand_Match,TMH577-hF-016-C12,TMH577-hIgG1-013-F3_F03,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCGAGAGGAGTGACTACGGCCGACTAC,"(G, G, G, C, C, T, G, C, C, T, A, G, C, C, A, ...","(G, G, G, C, C, T, G, C, C, T, A, G, C, C, A, ..."
4,Forward_Strand_Match,TMH577-hF-017-C01,TMH577-hIgG1-013-G3_G03,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCGAATCTGTCTGCGTGGGAAGTACCC,"(G, G, G, G, A, A, A, G, G, C, C, T, G, G, G, ...","(G, G, G, G, A, A, A, G, G, C, C, T, G, G, G, ..."


In [12]:
df_vh.head()

,Match,h3_name,sample_id,vh_abi_fp,vl_abi_fp,probe_seq,abi_seq,seq_record
0,Reverse_Strand_Match,TMH577-hF-012-E03,TMH577-hIgG1-013-A10,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCGAGACCCTTTTGGAGCGGCCTAACAAGAGAAAACTACTATTACG...,"(T, C, A, C, G, G, G, G, A, T, T, T, C, C, A, ...","(A, T, G, C, C, A, A, A, G, C, C, C, A, A, G, ..."
1,Reverse_Strand_Match,TMH577-hF-015-C12,TMH577-hIgG1-013-A11,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCTAGGTACTACTACGGTATGGACGTC,"(T, T, A, T, G, G, G, A, C, T, T, T, C, C, T, ...","(C, C, G, T, T, C, C, A, A, A, A, C, C, A, A, ..."
2,Reverse_Strand_Match,TMH577-hF-014-B12,TMH577-hIgG1-013-A12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCCACACGCAACCCTCTTCCACGGCAATATTACTACTACTTCTACG...,"(T, T, T, C, T, C, T, C, C, A, C, A, G, G, T, ...","(T, A, T, A, G, G, G, C, C, C, T, T, G, G, G, ..."
3,Reverse_Strand_Match,TMH577-hF-015-C08,TMH577-hIgG1-013-A12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCCACACGCAACCCTCTTCCACGGCAATATTACTACTACTTCTACG...,"(T, T, T, C, T, C, T, C, C, A, C, A, G, G, T, ...","(T, A, T, A, G, G, G, C, C, C, T, T, G, G, G, ..."
4,Reverse_Strand_Match,TMH577-hF-015-D08,TMH577-hIgG1-013-A12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCCACACGCAACCCTCTTCCACGGCAATATTACTACTACTTCTACG...,"(T, T, T, C, T, C, T, C, C, A, C, A, G, G, T, ...","(T, A, T, A, G, G, G, C, C, C, T, T, G, G, G, ..."


In [13]:
df_vh.seq_record.iloc[10]

SeqRecord(seq=Seq('ACGGTCAAAGCCCCAAGGGTCAAGGCCTGGGCCTGCCCCAGAAAGCTTACTCAC...CAA'), id='40250105', name='TMH577-hIgG1-013-A5_A05_GATC-VH60-2617917.abi', description='', dbxrefs=[])

In [14]:
df_vl.seq_record.iloc[10]

SeqRecord(seq=Seq('GGGCAGGCTGAGGAAGCAGGGTCTGACAGGGAGGGCAGAGGTCCTTGCTCAGGA...CTT'), id='40250607', name='TMH577-hIgG1-014-C10_VL79.abi', description='', dbxrefs=[])

In [15]:
df_vh.shape, df_vl.shape

((183, 8), (23, 8))

In [16]:
log.info(f"\nCOPY MATCHED ABI FILES INTO NEW LOCATION")
res_df_copy = copy_mtched_abi_files_to_resdir(res_dir_vh,res_dir_vl, res_dir_vh, log, True, df_vh)
log.info(f"\nFINISH COPYING FILES")

### sanity check

In [17]:
res_df_copy.h3_name.value_counts()

TMH577-hF-014-B12    22
TMH577-hF-015-C08    22
TMH577-hF-015-D08    22
TMH577-hF-017-G04    22
TMH577-hF-012-E03    14
                     ..
TMH577-hF-016-D06     2
TMH577-hF-012-C01     2
TMH577-hF-016-C03     2
TMH577-hF-004-G09     2
TMH577-hF-017-D03     2
Name: h3_name, Length: 67, dtype: int64

In [18]:
res_df_copy

,h3_name,sample_id,abi_out_loc,abi_initial_filepath
0,TMH577-hF-012-E03,TMH577-hIgG1-013-A10,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
1,TMH577-hF-012-E03,TMH577-hIgG1-013-H3_H03,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
2,TMH577-hF-012-E03,TMH577-hIgG1-013-H7_H07,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
3,TMH577-hF-012-E03,TMH577-hIgG1-014-A5_A05,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
4,TMH577-hF-012-E03,TMH577-hIgG1-014-C8_C08,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
...,...,...,...,...
178,TMH577-hF-016-E02,TMH577-hIgG1-014-E11,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
179,TMH577-hF-014-B02,TMH577-hIgG1-014-F2_F02,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
180,TMH577-hF-014-B02,TMH577-hIgG1-014-H2_H02,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
181,TMH577-hF-017-B02,TMH577-hIgG1-014-G12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...


In [19]:
res_df_copy[res_df_copy.sample_id == 'TMH577-hIgG1-013-H7_H07'].iloc[0]["abi_initial_filepath"]

'/Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-013-H7_H07_GATC-VH60-2617917.abi'

In [20]:
res_df_copy[res_df_copy.sample_id == 'TMH577-hIgG1-013-H7_H07'].iloc[1]["abi_initial_filepath"]

'/Users/rp/Desktop/purge/ny-immuno-sanger/data02/AB1_seqs/TMH577-hIgG1-013-H7_H07_VL79.abi'

### Note
- todo
    - create a dict for template_gp_for_vh and vl
    - copy the abi files to a new dir


In [21]:
# for template_strand in os.listdir(vh_template_sequence_folder):
#     parsed_template_strand = template_strand.split('VH-')[1].split('.gb')[0]
#     print(f"{template_strand} | {parsed_template_strand}")

In [22]:
sample_ids[0:10]

['TMH577-hIgG1-013-A10',
 'TMH577-hIgG1-013-A11',
 'TMH577-hIgG1-013-A12',
 'TMH577-hIgG1-013-A1_A01',
 'TMH577-hIgG1-013-A2_A02',
 'TMH577-hIgG1-013-A3_A03',
 'TMH577-hIgG1-013-A4_A04',
 'TMH577-hIgG1-013-A5_A05',
 'TMH577-hIgG1-013-A6_A06',
 'TMH577-hIgG1-013-A7_A07']

In [23]:
df_vh.head(3)

,Match,h3_name,sample_id,vh_abi_fp,vl_abi_fp,probe_seq,abi_seq,seq_record
0,Reverse_Strand_Match,TMH577-hF-012-E03,TMH577-hIgG1-013-A10,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCGAGACCCTTTTGGAGCGGCCTAACAAGAGAAAACTACTATTACG...,"(T, C, A, C, G, G, G, G, A, T, T, T, C, C, A, ...","(A, T, G, C, C, A, A, A, G, C, C, C, A, A, G, ..."
1,Reverse_Strand_Match,TMH577-hF-015-C12,TMH577-hIgG1-013-A11,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCTAGGTACTACTACGGTATGGACGTC,"(T, T, A, T, G, G, G, A, C, T, T, T, C, C, T, ...","(C, C, G, T, T, C, C, A, A, A, A, C, C, A, A, ..."
2,Reverse_Strand_Match,TMH577-hF-014-B12,TMH577-hIgG1-013-A12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCCACACGCAACCCTCTTCCACGGCAATATTACTACTACTTCTACG...,"(T, T, T, C, T, C, T, C, C, A, C, A, G, G, T, ...","(T, A, T, A, G, G, G, C, C, C, T, T, G, G, G, ..."


In [24]:
df_vh[df_vh.h3_name == 'TMH577-hF-004-G09']

,Match,h3_name,sample_id,vh_abi_fp,vl_abi_fp,probe_seq,abi_seq,seq_record
9,Reverse_Strand_Match,TMH577-hF-004-G09,TMH577-hIgG1-013-A4_A04,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,GCGAGACTTAACACCTGGGGCGGTGTGGACGTC,"(C, A, A, T, G, G, G, C, G, T, G, G, A, T, A, ...","(C, C, G, G, T, C, C, A, A, A, G, C, A, A, A, ..."


In [40]:
# This loop iterate through the genbank file and try matching the gb sample name with h3_name in the df_vh ( which is matched against the prob) 

gb_match_with_dfvh_lst = []
for gb_id in [*vh_template_gb_dict.keys()]:
    _ = df_vh[df_vh.h3_name == gb_id.replace('VH-', '')]
    if not (_).empty:
        for j in range(_.shape[0]):
            # print(f"Matched | {gb_id} | {_.iloc[j].h3_name} | {_.iloc[j].sample_id} | {vh_template_gb_dict.get(gb_id)}  | {_.iloc[j].vh_abi_fp} | {_.iloc[j].vl_abi_fp}")
            gb_match_with_dfvh_lst.append(['GB_Matched',gb_id,  _.iloc[j].h3_name, _.iloc[j].sample_id, vh_template_gb_dict.get(gb_id), _.iloc[j].vh_abi_fp, _.iloc[j].vl_abi_fp])
    else:
        pass
        # print(f"Not Matched {gb_id}")
        
gb_h3_matched_df = pd.DataFrame(gb_match_with_dfvh_lst)
gb_h3_matched_df.columns = ["Match", "gb_id", "h3_name", "sample_id", "gb_fp", "vh_abi_fp", "vl_abi_fp"]

In [35]:
# def find_gb_match_on_all_h3probes(gb_tmplate_dict: dict, df_vh: pd.DataFrame, pattern='VH-') -> pd.DataFrame:
#     gb_match_with_dfvl_lst = []
#     for gb_id in [*gb_tmplate_dict.keys()]:
#         _ = df_vh[df_vh.h3_name == gb_id.replace(pattern, '')]
#         if not (_).empty:
#             for j in range(_.shape[0]):
#                 # print(f"Matched|{gb_id}|{_.iloc[j].h3_name}|{_.iloc[j].sample_id}|{gb_tmplate_dict.get(gb_id)}|{_.iloc[j].vh_abi_fp}|{_.iloc[j].vl_abi_fp}")
#                 gb_match_with_dfvl_lst.append(['GB_Matched',gb_id, _.iloc[j].h3_name, _.iloc[j].sample_id, gb_tmplate_dict.get(gb_id), _.iloc[j].vh_abi_fp, _.iloc[j].vl_abi_fp])
#             # print()
#         else:
#             pass
#             # print(f"Not Matched {gb_id}")
#     _df = pd.DataFrame(gb_match_with_dfvh_lst)
#     _df.columns = ["Match", "gb_id", "h3_name", "sample_id", "gb_fp", "vh_abi_fp", "vl_abi_fp"]
#     return _df

# find_gb_match_on_all_h3probes(vl_template_gb_dict, df_vh, pattern='VL-')

In [39]:
gb_h3_matched_df

,Match,gb_id,h3_name,sample_id,gb_fp,vh_abi_fp,vl_abi_fp
0,GB_Matched,VH-TMH577-hF-004-G09,TMH577-hF-004-G09,TMH577-hIgG1-013-A4_A04,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
1,GB_Matched,VH-TMH577-hF-005-G08,TMH577-hF-005-G08,TMH577-hIgG1-014-A8_A08,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
2,GB_Matched,VH-TMH577-hF-005-G08,TMH577-hF-005-G08,TMH577-hIgG1-014-C4_C04,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
3,GB_Matched,VH-TMH577-hF-012-A02,TMH577-hF-012-A02,TMH577-hIgG1-013-F12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
4,GB_Matched,VH-TMH577-hF-012-A02,TMH577-hF-012-A02,TMH577-hIgG1-014-G9_G09,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
...,...,...,...,...,...,...,...
178,GB_Matched,VH-TMH577-hF-017-H02,TMH577-hF-017-H02,TMH577-hIgG1-013-B12,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
179,GB_Matched,VH-TMH577-hF-017-H02,TMH577-hF-017-H02,TMH577-hIgG1-014-H4_H04,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
180,GB_Matched,VH-TMH577-hF-017-H03,TMH577-hF-017-H03,TMH577-hIgG1-013-A2_A02,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
181,GB_Matched,VH-TMH577-hF-017-H03,TMH577-hF-017-H03,TMH577-hIgG1-013-C10,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...,/Users/rp/Desktop/purge/ny-immuno-sanger/data0...
